#### 1. Import pandas library

In [1]:
import pandas as pd

#### 2. Import users table:

In [2]:
users=pd.read_csv(r"C:\Users\Alina Hartsch\OneDrive\Desktop\Ironhack\My_Lab_Solutions\Week 2\lab-data-cleaning\your-code\users_table.csv")
users.head(3)

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,NaN


#### 3. Rename Id column to userId

In [3]:
users=users.rename(columns = {'Id':'userId'})
users.head(3)

,userId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,NaN


#### 4. Import posts table:

In [4]:
posts=pd.read_csv(r"C:\Users\Alina Hartsch\OneDrive\Desktop\Ironhack\My_Lab_Solutions\Week 2\lab-data-cleaning\your-code\posts_table.csv")
posts.head(3)

,Id,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,OwnerUserId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaN,NaN,NaN


#### 5. Rename Id column to postId and OwnerUserId to userId

In [5]:
posts=posts.rename(columns = {'Id':'postId', 'OwnerUserId':'userId'})
posts.head(3)

,postId,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,userId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaN,NaN,NaN


#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [6]:

# Create New DataFrame of Specific column by DataFrame.assign() method.
users2 = pd.DataFrame().assign(userID=users['userId'], Reputation=users['Reputation'], Views=users['Views'], UpVotes=users['UpVotes'], DownVotes=users['DownVotes'])
users2.head(3)


,userID,Reputation,Views,UpVotes,DownVotes
0,-1,1,0,5007,1920
1,2,101,25,3,0
2,3,101,22,19,0


In [7]:
posts2 = pd.DataFrame().assign(postID=posts['postId'], Score=posts['Score'], userId=posts['userId'], ViewCount=posts['ViewCount'], CommentCount=posts['CommentCount'])
posts2.head(3)

,postID,Score,userId,ViewCount,CommentCount
0,1,23,8.0,1278.0,1
1,2,22,24.0,8198.0,1
2,3,54,18.0,3613.0,4


#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [38]:
merged_df = pd.merge(left = users2, right = posts2, how = 'outer', left_on = 'userID', right_on = 'userId')
merged_df

,userID,Reputation,Views,UpVotes,DownVotes,postID,Score,userId,ViewCount,CommentCount
0,-1.0,1.0,0.0,5007.0,1920.0,2175.0,0.0,-1.0,NaN,0.0
1,-1.0,1.0,0.0,5007.0,1920.0,8576.0,0.0,-1.0,NaN,0.0
2,-1.0,1.0,0.0,5007.0,1920.0,8578.0,0.0,-1.0,NaN,0.0
3,-1.0,1.0,0.0,5007.0,1920.0,8981.0,0.0,-1.0,NaN,0.0
4,-1.0,1.0,0.0,5007.0,1920.0,8982.0,0.0,-1.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...
72182,NaN,NaN,NaN,NaN,NaN,46117.0,0.0,NaN,355.0,0.0
72183,NaN,NaN,NaN,NaN,NaN,46260.0,5.0,NaN,145.0,5.0
72184,NaN,NaN,NaN,NaN,NaN,46836.0,3.0,NaN,406.0,0.0
72185,NaN,NaN,NaN,NaN,NaN,46892.0,1.0,NaN,NaN,0.0


#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [39]:
merged_df.isnull().sum()

userID           1038
Reputation       1038
Views            1038
UpVotes          1038
DownVotes        1038
postID          32187
Score           32187
userId          33225
ViewCount       56292
CommentCount    32187
dtype: int64

#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

I will fill the NaN Values of each Column with a 0 besides the ViewCount one since ViewCount has almost 70% nones

In [49]:
merged_df[['userID', 'Reputation', 'Views', 'UpVotes', 'DownVotes']]=merged_df[['userID', 'Reputation', 'Views', 'UpVotes', 'DownVotes']].fillna(0)
merged_df[['postID', 'Score', 'userId', 'CommentCount']]=merged_df[['postID', 'Score', 'userId', 'CommentCount']].fillna(0)
merged_df=merged_df.drop(columns='ViewCount')
merged_df.isnull().sum()

userID          0
Reputation      0
Views           0
UpVotes         0
DownVotes       0
postID          0
Score           0
userId          0
CommentCount    0
dtype: int64

#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [50]:
merged_df.dtypes

userID          float64
Reputation      float64
Views           float64
UpVotes         float64
DownVotes       float64
postID          float64
Score           float64
userId          float64
CommentCount    float64
dtype: object

I would change everything besides the Score & Reputation from float to int since float does not make sense because you can't have 1/2 view for example

In [52]:
merged_df[['userID','Views', 'UpVotes', 'DownVotes', 'postID', 'userId', 'CommentCount']]=merged_df[['userID','Views', 'UpVotes', 'DownVotes', 'postID', 'userId', 'CommentCount']].astype('int64')

In [53]:
merged_df.dtypes

userID            int64
Reputation      float64
Views             int64
UpVotes           int64
DownVotes         int64
postID            int64
Score           float64
userId            int64
CommentCount      int64
dtype: object